imports and data uploads
-----------

In [46]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.simplefilter('ignore')
import os
%matplotlib inline
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.preprocessing import OneHotEncoder
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.tools.tools import pinv_extended  
from statsmodels.stats.anova import anova_lm
from tqdm.auto import tqdm

In [47]:
# upload data
cpath = os.getcwd() #assumes mean_df_females is in the same path as the notebook
female_mean_df = pd.read_csv(cpath+'/mean_df_female.csv')
female_scalar_df = pd.read_csv(cpath+'/scalar_df_female.csv')
male_mean_df = pd.read_csv(cpath+'/mean_df_male.csv')
male_scalar_df = pd.read_csv(cpath+'/scalar_df_male.csv')

# ----------------------------------------------------------------------- 
# ANALYSIS
# -----------------------------------------------------------------------

In [48]:
def cal_scalars(mean_df,scalar_df,metric='robust'):
    # area, height, width, length
    if metric=='robust':
        hdf = scalar_df.groupby('mouse')['height_ave_mm'].quantile(0.95) - scalar_df.groupby('mouse')['height_ave_mm'].quantile(0.05)
        wdf = scalar_df.groupby('mouse')['width_mm'].quantile(0.95) - scalar_df.groupby('mouse')['width_mm'].quantile(0.05)
        adf = scalar_df.groupby('mouse')['area_mm'].quantile(0.95) - scalar_df.groupby('mouse')['area_mm'].quantile(0.05)
        ldf = scalar_df.groupby('mouse')['length_mm'].quantile(0.95) - scalar_df.groupby('mouse')['length_mm'].quantile(0.05)
    
    if metric=='range':
        hdf = scalar_df.groupby('mouse')['height_ave_mm'].max() - scalar_df.groupby('mouse')['height_ave_mm'].min()
        wdf = scalar_df.groupby('mouse')['width_mm'].max() - scalar_df.groupby('mouse')['width_mm'].min()
        adf = scalar_df.groupby('mouse')['area_mm'].max() - scalar_df.groupby('mouse')['area_mm'].min()
        ldf = scalar_df.groupby('mouse')['length_mm'].max() - scalar_df.groupby('mouse')['length_mm'].min()
    
    if metric=='max':
        hdf = scalar_df.groupby('mouse')['height_ave_mm'].max()
        wdf = scalar_df.groupby('mouse')['width_mm'].max()
        adf = scalar_df.groupby('mouse')['area_mm'].max()
        ldf = scalar_df.groupby('mouse')['length_mm'].max()
    
    if metric=='min':
        hdf = scalar_df.groupby('mouse')['height_ave_mm'].min()
        wdf = scalar_df.groupby('mouse')['width_mm'].min()
        adf = scalar_df.groupby('mouse')['area_mm'].min()
        ldf = scalar_df.groupby('mouse')['length_mm'].min()
        
    if metric=='mean':
        hdf = scalar_df.groupby('mouse')['height_ave_mm'].mean()
        wdf = scalar_df.groupby('mouse')['width_mm'].mean()
        adf = scalar_df.groupby('mouse')['area_mm'].mean()
        ldf = scalar_df.groupby('mouse')['length_mm'].mean()
        
    #syllable usage
    temp_mean_df = mean_df.groupby(by = ['mouse','syllable']).mean()['usage'].reset_index()
    mean_df_lc = pd.pivot_table(temp_mean_df, values='usage', index=['mouse'], columns=['syllable']).reset_index().fillna(0)
    syll =mean_df_lc.drop(['mouse'], axis=1).to_numpy()
    syll = np.vstack(syll)

    temp_mean_df['height']=0
    temp_mean_df['width']=0
    temp_mean_df['area']=0
    temp_mean_df['length']=0

    mice = sorted(mean_df.mouse.unique())
    for i,m in enumerate(mice):
        temp_mean_df['height'][temp_mean_df.mouse == m] = hdf.values[i]

    for i,m in enumerate(mice):
        temp_mean_df['width'][temp_mean_df.mouse == m] = wdf.values[i]

    for i,m in enumerate(mice):
        temp_mean_df['area'][temp_mean_df.mouse == m] = adf.values[i]

    for i,m in enumerate(mice):
        temp_mean_df['length'][temp_mean_df.mouse == m] = ldf.values[i]
    return temp_mean_df

In [49]:
female_scalars = cal_scalars(female_mean_df,female_scalar_df,'robust')
male_scalars = cal_scalars(male_mean_df,male_scalar_df,'robust')
scalars = ['height','width','area','length']

# GLM to predict size

In [50]:
female_syll=[]
male_syll=[]

## for females

In [51]:
syll_data = pd.pivot_table(female_scalars, values='usage', index=['mouse'], columns=['syllable']).reset_index().fillna(0)
scalars_data = female_scalars.groupby(['mouse']).mean().drop(['syllable','usage'], axis=1)
syll = female_scalars.syllable.unique()

#prepare syllable data
y = syll_data.drop(['mouse'], axis=1).to_numpy()
y_log = np.log(y + 1e-6)
x = (y_log - y_log.mean(axis=0, keepdims=True)) / y_log.std(axis=0, keepdims=True)

In [52]:
corr_syll=[]
for s in scalars:
    # run GLM on actual data
    y = scalars_data[s].to_numpy().reshape(-1, 1)
    lr = ElasticNet(alpha=0.01)
    lr.fit(x, y)
    print(s,'model fit ',lr.score(x, y))
    data_scores = lr.coef_ # score per syllbable
    
    # shuffle size
    it =1000
    shuff_scores=[]
    for i in range(it):
        newy = np.random.permutation(y)
        lr.fit(x,newy)
        shuff_scores.append(lr.coef_)

    threshh = np.quantile(shuff_scores,.975,axis=0) 
    threshl = np.quantile(shuff_scores,.025,axis=0) 
    corr_syll.append(syll[(data_scores>threshh) | (data_scores<threshl)]) # add syllables that pass threshold
female_syll = np.unique(np.concatenate(corr_syll))
print('size correlated syllables for females: ',female_syll)

height model fit  0.9993047495655724
width model fit  0.9981311240822363
area model fit  0.9999315778204688
length model fit  0.9984083395781147
size correlated syllables for females:  [ 0 13 19 23 25 27 35 36 46]


## for males

In [53]:
syll_data = pd.pivot_table(male_scalars, values='usage', index=['mouse'], columns=['syllable']).reset_index().fillna(0)
scalars_data = male_scalars.groupby(['mouse']).mean().drop(['syllable','usage'], axis=1)
syll = male_scalars.syllable.unique()

#prepare syllable data
y = syll_data.drop(['mouse'], axis=1).to_numpy()
y_log = np.log(y + 1e-6)
x = (y_log - y_log.mean(axis=0, keepdims=True)) / y_log.std(axis=0, keepdims=True)

In [54]:
corr_syll=[]
for s in scalars:
    # run GLM on actual data
    y = scalars_data[s].to_numpy().reshape(-1, 1)
    lr = ElasticNet(alpha=0.01)
    lr.fit(x, y)
    print(s,'model fit ',lr.score(x, y))
    data_scores = lr.coef_ # score per syllbable
    # shuffle size
    it =1000
    shuff_scores=[]
    for i in range(it):
        newy = np.random.permutation(y)
        lr.fit(x,newy)
        shuff_scores.append(lr.coef_)

    threshh = np.quantile(shuff_scores,.975,axis=0) 
    threshl = np.quantile(shuff_scores,.025,axis=0) 
    corr_syll.append(syll[(data_scores>threshh) | (data_scores<threshl)]) # add syllables that pass threshold
male_syll = np.unique(np.concatenate(corr_syll))
print('size correlated syllables for males: ',male_syll)

height model fit  0.9990634758590967
width model fit  0.9936843527096059
area model fit  0.9998366713614902
length model fit  0.9991668705985325
size correlated syllables for males:  [ 2  9 15 33 46]


## control for co-lineratity

## for females

In [55]:
data=female_scalars
keep_syll = list(range(50))
keep_syll = [ele for ele in keep_syll if ele not in female_syll]
data=data[data.syllable.isin(keep_syll)]
syll_data = pd.pivot_table(data, values='usage', index=['mouse'], columns=['syllable']).reset_index().fillna(0)
scalars_data = data.groupby(['mouse']).mean().drop(['syllable','usage'], axis=1)
syll = data.syllable.unique()

#prepare syllable data
y = syll_data.drop(['mouse'], axis=1).to_numpy()
y_log = np.log(y + 1e-6)
x = (y_log - y_log.mean(axis=0, keepdims=True)) / y_log.std(axis=0, keepdims=True)

In [56]:
corr_syll=[]
for s in scalars:
    # run GLM on actual data
    y = scalars_data[s].to_numpy().reshape(-1, 1)
    lr = ElasticNet(alpha=0.01)
    lr.fit(x, y)
    print(s,'model fit ',lr.score(x, y))
    data_scores = lr.coef_ # score per syllbable    
    # shuffle size
    it =1000
    shuff_scores=[]
    for i in range(it):
        newy = np.random.permutation(y)
        lr.fit(x,newy)
        shuff_scores.append(lr.coef_)

    threshh = np.quantile(shuff_scores,.975,axis=0) 
    threshl = np.quantile(shuff_scores,.025,axis=0) 
    corr_syll.append(syll[(data_scores>threshh) | (data_scores<threshl)]) # add syllables that pass threshold
female_syll2 = np.unique(np.concatenate(corr_syll))
print('additional size correlated syllables for females: ',female_syll2)

height model fit  0.9990051391051373
width model fit  0.9958390577075106
area model fit  0.9997705135707747
length model fit  0.9980309873978669
additional size correlated syllables for females:  [ 2 10 16 38 40 47]


## for males

In [57]:
data=male_scalars
keep_syll = list(range(50))
keep_syll = [ele for ele in keep_syll if ele not in male_syll]
data=data[data.syllable.isin(keep_syll)]
syll_data = pd.pivot_table(data, values='usage', index=['mouse'], columns=['syllable']).reset_index().fillna(0)
scalars_data = data.groupby(['mouse']).mean().drop(['syllable','usage'], axis=1)
syll = data.syllable.unique()

#prepare syllable data
y = syll_data.drop(['mouse'], axis=1).to_numpy()
y_log = np.log(y + 1e-6)
x = (y_log - y_log.mean(axis=0, keepdims=True)) / y_log.std(axis=0, keepdims=True)

In [58]:
corr_syll=[]
for s in scalars:
    # run GLM on actual data
    y = scalars_data[s].to_numpy().reshape(-1, 1)
    lr = ElasticNet(alpha=0.01)
    lr.fit(x, y)
    print(s,'model fit ',lr.score(x, y))
    data_scores = lr.coef_ # score per syllbable
  
    # shuffle size
    it =1000
    shuff_scores=[]
    for i in range(it):
        newy = np.random.permutation(y)
        lr.fit(x,newy)
        shuff_scores.append(lr.coef_)

    threshh = np.quantile(shuff_scores,.975,axis=0) 
    threshl = np.quantile(shuff_scores,.025,axis=0) 
    corr_syll.append(syll[(data_scores>threshh) | (data_scores<threshl)]) # add syllables that pass threshold
male_syll2 = np.unique(np.concatenate(corr_syll))
print('additional size correlated syllables for males: ',male_syll2)

height model fit  0.999224867877009
width model fit  0.9910989131486606
area model fit  0.9997241295362839
length model fit  0.9991516873309965
additional size correlated syllables for males:  [18 27 39]


In [59]:
## syllables
male_syll=np.concatenate([male_syll,male_syll2])
female_syll=np.concatenate([female_syll,female_syll2])

In [60]:
print('size correlated syllables for males: ',sorted(male_syll))
print('size correlated syllables for females: ',sorted(female_syll))

size correlated syllables for males:  [2, 9, 15, 18, 27, 33, 39, 46]
size correlated syllables for females:  [0, 2, 10, 13, 16, 19, 23, 25, 27, 35, 36, 38, 40, 46, 47]
